In [ ]:
import pandas as pd

In [ ]:
# Define the column names for the UNSW-NB15 dataset
column_names = [
    'srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl',
    'sloss', 'dloss', 'service', 'sload', 'dload', 'spkts', 'dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb',
    'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len', 'sjit', 'djit', 'stime', 'ltime', 'sinpkt',
    'dinpkt', 'tcprtt', 'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd',
    'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm',
    'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat', 'label'
]

#Load the datasets without headers
df1 = pd.read_csv('Datasets/UNSW-NB15/UNSW-NB15_1.csv', header=None, names=column_names, low_memory=False)
df2 = pd.read_csv('Datasets/UNSW-NB15/UNSW-NB15_2.csv', header=None, names=column_names, low_memory=False)
df3 = pd.read_csv('Datasets/UNSW-NB15/UNSW-NB15_3.csv', header=None, names=column_names, low_memory=False)
df4 = pd.read_csv('Datasets/UNSW-NB15/UNSW-NB15_4.csv', header=None, names=column_names, low_memory=False)

#Combine into one dataframe
df = pd.concat([df1, df2, df3, df4])

print(df.shape)
print(df.head())

In [ ]:
# Exploratory Data Analysis
print(df.info())

# Plotly visualization of the 'proto' column
import plotly.express as px

proto_counts = df['proto'].value_counts()
fig = px.bar(x=proto_counts.index, y=proto_counts.values, title='Distribution of Protocols')
fig.show()